### Vector Stores and Retrievers

The familiarize you with LangChain's vector store and retriever abstractions. These abstrations are designed to support retrieval of data from (vector) databases and other sources for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

* Documents
* Vector Stores
* Retrievers

In [ ]:
langchain==0.3.26
langchain-groq==0.3.4
langchain-chroma==0.1.4
langchain-huggingface==0.2.0
pydantic==2.11.7

### Documents

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It as two attributes:

- page_content: a string representing the content.
- metadata: a dict containing arbitrary metadata.
    The metadata attribute can capture information about the source of the document, its relationship to other documents and other information. Note that an individual Document object often represents a chunk of a larger document.



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq

groq_api_key = os.getenv('GROQ_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

llm = ChatGroq(groq_api_key=groq_api_key, model='llama3-8b-8192')

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2') # Sentence 

c:\Users\LENOVO\OneDrive\Desktop\Project\GenAI\LangChain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

from langchain_core.documents import Document

# Load the document
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Gold fish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"}
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"}
    )
]

In [9]:
# Vector Stores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=documents, embedding=embeddings)

In [10]:
# Await
await vectorstore.asimilarity_search('cat')

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Gold fish are popular pets for beginners, requiring relatively simple care.')]

In [11]:
# Score
vectorstore.similarity_search_with_score('cat')

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351057410240173),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.5956902503967285),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Gold fish are popular pets for beginners, requiring relatively simple care.'),
  1.6560044288635254)]

### Retrievers

LangChain VectorStore objects do not sub class Runnable and so cannot immediately be integrated into LangChain Expression Language Chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without sub classing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method

In [12]:
# Runnable Lambda - Retriever
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1) # k top value
retriever.batch(['cat', 'dog'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

##### Vectore Store - Retriever

Vector stores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vectore store to call and how to parameterize them. For instance, we can replicate the above with following:

In [13]:
retriver_vectore = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriver_vectore.batch(['cat', 'dog'])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [22]:
from langchain_core.prompts import ChatPromptTemplate

message = """
    Answer this question using the provided context only.
    
    {question}
    
    Context:
    {context}
"""

prompt = ChatPromptTemplate.from_messages([('human', message)])

rag_chain = {"context": retriver_vectore, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke('tell me about rat')
print(response.content)


I apologize, but there is no information about rats in the provided context. The context only talks about rabbits being social animals that need space to hop around. There is no mention of rats.
